In [25]:
import csv
import math
import numpy as np
import pandas as pd
#libs = dict()
#apis = dict()
#with open("export/lib.csv", "r") as f:
#    reader = csv.DictReader(f)
#    for row in reader:
#        row["signatureIds"] = row["signatureIds"].split(";")
#        libs[row["groupArtifactId"]] = row
#with open("export/api.csv", "r") as f:
#    reader = csv.DictReader(f)
#    for row in reader:
#        apis[row["signatureId"]] = row
libs = pd.read_csv("../export/Lib.csv", ).set_index("groupArtifactId")
apis = pd.read_csv("../export/API.csv", dtype={"signatureId":str}).set_index("signatureId")
libs["signatureIds"] = libs["signatureIds"].apply(lambda x: x.split(";") if type(x) == str else [])
apis["paramList"] = apis["paramList"].apply(lambda x: x.split(";") if type(x) == str else [])

In [26]:
libs.head()

,groupArtifactName,signatureIds
groupArtifactId,,
155,org.junit.jupiter:junit-jupiter-engine,"[1958505103816, 1958505103817, 1958505103818, ..."
61267,commons-discovery:commons-discovery,"[240519190535, 240519190534, 240519190533, 240..."
90,org.assertj:assertj-core,"[1786706395136, 1786706395137, 1786706395138, ..."
83,org.hibernate:hibernate-entitymanager,"[1271310325070, 1271310325071, 2748779075311, ..."
382,org.xerial:sqlite-jdbc,"[2851858334364, 2851858334365, 2851858334366, ..."


In [27]:
apis.head()

,packageName,className,methodName,paramList
signatureId,,,,
2061584302560,com.google.common.collect,ArrayListMultimap,<init>,[]
1305670058288,com.google.common.collect,AbstractMultimap$SortedAsMap,headMap,[java.lang.Object]
3710851744608,com.google.common.collect,Lists$TransformingSequentialList,replaceAll,[java.util.function.UnaryOperator]
2714419331704,com.google.common.base,CharMatcher$6,clone,[]
3195455668968,com.google.common.collect,ForwardingIterator,wait,[]


In [60]:
print(len(set(libs["groupArtifactName"])))
print(set(libs["groupArtifactName"]))

109
{'org.eclipse.persistence:eclipselink', 'commons-discovery:commons-discovery', 'commons-io:commons-io', 'org.hibernate:hibernate-core', 'org.slf4j:slf4j-simple', 'org.mockito:mockito-all', 'org.apache.commons:commons-collections4', 'org.seleniumhq.selenium:selenium-server', 'net.sf.jasperreports:jasperreports', 'org.hamcrest:java-hamcrest', 'asm:asm', 'net.sourceforge.jtds:jtds', 'org.slf4j:slf4j-nop', 'org.reflections:reflections', 'org.apache.derby:derby', 'org.codehaus.groovy:groovy', 'it.unimi.dsi:fastutil', 'com.fasterxml.jackson.core:jackson-annotations', 'org.powermock:powermock-module-junit4', 'javassist:javassist', 'org.apache.httpcomponents:httpmime', 'xerces:xercesImpl', 'org.apache.logging.log4j:log4j-core', 'org.testng:testng', 'org.easymock:easymock', 'org.apache.logging.log4j:log4j-api', 'org.codehaus.jackson:jackson-mapper-asl', 'org.codehaus.jackson:jackson-xc', 'freemarker:freemarker', 'hsqldb:hsqldb', 'easymock:easymock', 'velocity:velocity', 'joda-time:joda-time

In [29]:
print(libs[libs["signatureIds"].apply(lambda x: len(x) == 0)])

                        groupArtifactName signatureIds
groupArtifactId                                       
284              net.sf.json-lib:json-lib           []


In [63]:
def get_signatures(libname):
    signatureIds = libs[libs["groupArtifactName"] == libname]["signatureIds"].iloc[0]
    return apis.loc[signatureIds]
def to_classes(signatures):
    signatures["packageName"] = signatures["packageName"].apply(lambda x: x if type(x) == str else "")
    return sorted(list(set(signatures["packageName"] + "." + signatures["className"])))
def to_packages(signatures):
    signatures["packageName"] = signatures["packageName"].apply(lambda x: x if type(x) == str else "")
    return sorted(list(set(signatures["packageName"])))
to_classes(get_signatures("org.json:json"))

['org.json.CDL',
 'org.json.Cookie',
 'org.json.CookieList',
 'org.json.HTTP',
 'org.json.HTTPTokener',
 'org.json.JSONArray',
 'org.json.JSONException',
 'org.json.JSONML',
 'org.json.JSONObject',
 'org.json.JSONObject$1',
 'org.json.JSONObject$Null',
 'org.json.JSONPointer',
 'org.json.JSONPointer$Builder',
 'org.json.JSONPointerException',
 'org.json.JSONPropertyIgnore',
 'org.json.JSONPropertyName',
 'org.json.JSONString',
 'org.json.JSONStringer',
 'org.json.JSONTokener',
 'org.json.JSONWriter',
 'org.json.Kim',
 'org.json.Property',
 'org.json.Test',
 'org.json.Test$1Obj',
 'org.json.XML',
 'org.json.XML$1',
 'org.json.XML$1$1',
 'org.json.XMLParserConfiguration',
 'org.json.XMLTokener',
 'org.json.zip.BitInputStream',
 'org.json.zip.BitOutputStream',
 'org.json.zip.BitReader',
 'org.json.zip.BitWriter',
 'org.json.zip.Compressor',
 'org.json.zip.Decompressor',
 'org.json.zip.Huff',
 'org.json.zip.Huff$Symbol',
 'org.json.zip.JSONzip',
 'org.json.zip.Keep',
 'org.json.zip.MapKeep

In [64]:
to_packages(get_signatures("asm:asm"))

['org.objectweb.asm', 'org.objectweb.asm.signature']

In [68]:
# Export to csv for further analysis
from collections import OrderedDict
df = []
libnames = set(libs["groupArtifactName"])
for name in libnames:
    df.append(OrderedDict([
        ["groupArtifactName", name],
        ["hasProblem", ""],
        ["problemDescription", ""],
        ["packages", ";".join(to_packages(get_signatures(name)))]
    ]))
pd.DataFrame(df).to_csv("lib2pkg.csv", index=False)